In [ ]:
from dataset import SolarPanelDataset
from pathlib import Path
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
data_path = Path("../data")
batch_size = 128
lr = 0.0001
epochs = 25
seed = 1048596
probs = [0.8, 0.1, 0.1]
num_workers = 8
epsilon = 1e-7
prediction_threshold = 0.5
data_threshold = 0.01

chunk_size = 10

mode = "cls"

xlsx_path = data_path / "solar_panel_data_madagascar.xlsx"
img_path = data_path / "img"
weights_path = ""
tb_path = ""

torch.manual_seed(seed)

if mode == "seg":
    val_dataset = SolarPanelDataset(
        img_path, xlsx_path, "seg", "pan", "val", probs, seed
    )

elif mode == "cls":
    val_dataset = SolarPanelDataset(
        img_path, xlsx_path, "cls", "all", "val", probs, seed, data_threshold
    )

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=num_workers,
    persistent_workers=True,
)

torch.set_float32_matmul_precision("high")
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load(seg_weights_path, weights_only=False, map_location=device)
model = torch.compile(model).to(device)

loss_fn = torch.nn.BCEWithLogitsLoss()

activation = lambda x: (torch.nn.Sigmoid()(x) >= prediction_threshold).long()

if mode == "seg":
    metrics_dict = {"Jaccard": metrics.jaccard}

if mode == "cls":
    metrics_dict = {
        "F1": metrics.f1,
        "Accuracy": metrics.accuracy,
        "Precision": metrics.precision,
        "Recall": metrics.recall,
    }

val_metrics = train(
    model,
    mode,
    loss_fn,
    metrics,
    None,
    activation,
    1,
    device,
    None,
    val_dataloader,
    None,
    None,
    None,
    False,
    validation=True
)

In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd


def parse_tensorboard(path, scalars):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _absorb_print = ea.Reload()
    # make sure the scalars are in the event accumulator tags
    assert all(
        s in ea.Tags()["scalars"] for s in scalars
    ), "some scalars were not found in the event accumulator"
    return {k: pd.DataFrame(ea.Scalars(k)) for k in scalars}


In [ ]:
scalars = ["Loss/train", "Loss/test"]
for metric in metrics_dict:
    scalars.append(f"{metric}/train")
    scalars.append(f"{metric}/test")
parsed_tensorboard = parse_tensorboard(tb_path, scalars)

In [ ]:
metrics_list = list(metrics_dict.keys()) + ["Loss"]

for metric in metrics_dict:
    plt.plot(parsed_tensorboard[f"{metric}/test"]["step"], parsed_tensorboard[f"{metric}/test"]["value"], label="Test")
    plt.plot(parsed_tensorboard[f"{metric}/train"]["step"], parsed_tensorboard[f"{metric}/train"]["value"], label="Entraînement")
    plt.ylabel(f"{metric}")
    plt.xlabel("Epoch")
    plt.legend()
    plt.show()

In [ ]:
print(val_metrics)